# Managing Pydantic Data Models in FastAPI

In [1]:
import time
from pydantic import ValidationError
from chapter4_models import (
    Gender, Person1, Person2, Person3, Person4, Person5, Person6,
    UserProfile, User1, UserRegistration,
    ModelWithEarlyValidation, ModelWithBadDefaults, ModelWithGoodDefaults
)

A pydantic model composed of built-in types


In [2]:
person = Person1(first_name="John", last_name="Doe", age=30)
person

Person1(first_name='John', last_name='Doe', age=30)

A pydantic model composed of more complex types


In [3]:
# A person with an invalid gender

try:
    Person2(
        first_name="John",
        last_name="Doe",
        gender="INVALID_VALUE",
        birthdate="1991-01-01",
        interests=["travel", "sports"],
    )
except ValidationError as e:
    print(str(e))


1 validation error for Person2
gender
  value is not a valid enumeration member; permitted: 'MALE', 'FEMALE', 'NON_BINARY' (type=type_error.enum; enum_values=[<Gender.MALE: 'MALE'>, <Gender.FEMALE: 'FEMALE'>, <Gender.NON_BINARY: 'NON_BINARY'>])


In [4]:
# A person with an invalid birthdate

try:
    Person2(
        first_name="John",
        last_name="Doe",
        gender=Gender.MALE,
        birthdate="1991-13-42",
        interests=["travel", "sports"],
    )
except ValidationError as e:
    print(str(e))


1 validation error for Person2
birthdate
  invalid date format (type=value_error.date)


In [5]:
# A valid person

person = Person2(
    first_name="John",
    last_name="Doe",
    gender=Gender.MALE,
    birthdate="1991-01-01",
    interests=["travel", "sports"],
)
person

Person2(first_name='John', last_name='Doe', gender=<Gender.MALE: 'MALE'>, birthdate=datetime.date(1991, 1, 1), interests=['travel', 'sports'])

A pydantic model composed of another pydantic model as field

In [6]:
# A person with an invalid address

try:
    Person3(
        first_name="John",
        last_name="Doe",
        gender=Gender.MALE,
        birthdate="1991-01-01",
        interests=["travel", "sports"],
        address={
            "street_address": "12 Squirell Street",
            "postal_code": "424242",
            "city": "Woodtown",
            # Missing country
        },
    )
except ValidationError as e:
    print(str(e))


1 validation error for Person3
address -> country
  field required (type=value_error.missing)


In [7]:
# A person with a valid address

person = Person3(
    first_name="John",
    last_name="Doe",
    gender=Gender.MALE,
    birthdate="1991-01-01",
    interests=["travel", "sports"],
    address={
        "street_address": "12 Squirell Street",
        "postal_code": "424242",
        "city": "Woodtown",
        "country": "US",
    }, # A dict can be used to instantiate the model
)
person

Person3(first_name='John', last_name='Doe', gender=<Gender.MALE: 'MALE'>, birthdate=datetime.date(1991, 1, 1), interests=['travel', 'sports'], address=Address(street_address='12 Squirell Street', postal_code='424242', city='Woodtown', country='US'))

A pydantic model with optional fields


In [8]:
user = UserProfile(nickname="jdoe")
user


UserProfile(nickname='jdoe', location=None, subscribed_newsletter=True)

A pydantic model with an optional date field (not recommended)

In [9]:
# A default date field in a pydantic models is not a good idea,
# since a datetime field is evaluated once when the model is imported

obj1 = ModelWithBadDefaults() 
obj1.d # The default date field



datetime.datetime(2022, 7, 5, 10, 11, 7, 699862)

In [10]:
time.sleep(1)  # Wait for a second
obj2 = ModelWithBadDefaults()
obj2.d # The date field is the same, even if we create the 2nd object 1 second later!!


datetime.datetime(2022, 7, 5, 10, 11, 7, 699862)

In [11]:
obj1.d < obj2.d


False

A pydantic model that implements custom field validations

In [12]:
# A person with an invalid first name
try:
    Person4(first_name="J", last_name="Doe", age=30)
except ValidationError as e:
    print(str(e))


1 validation error for Person4
first_name
  ensure this value has at least 3 characters (type=value_error.any_str.min_length; limit_value=3)


In [13]:
# A person with an invalid age
try:
    Person4(first_name="John", last_name="Doe", age=2000)
except ValidationError as e:
    print(str(e))


1 validation error for Person4
age
  ensure this value is less than or equal to 120 (type=value_error.number.not_le; limit_value=120)


In [14]:
# A valid person

person = Person4(first_name="John", last_name="Doe", age=30)
person


Person4(first_name='John', last_name='Doe', age=30)

A pydantic model that allows dynamic default values

In [15]:
# Now dynamic fields such as date fields are created correctly, as follows

obj1 = ModelWithGoodDefaults()
print(obj1.l)
print(obj1.l2)
obj1.l.append("d")
print(obj1.l)
print(obj1.d)


['a', 'b', 'c']
[]
['a', 'b', 'c', 'd']
2022-07-05 10:11:13.175206


In [16]:
obj2 = ModelWithGoodDefaults()
print(obj2.l)
print(obj1.l2)
print(obj2.d)

['a', 'b', 'c']
[]
2022-07-05 10:11:13.286056


In [17]:
obj1.d < obj2.d  # True


True

A pydantic model that does validation of commom string types such as URLs and emails

In [18]:
# Invalid email

try:
    User1(email="jdoe", website="https://www.example.com")
except ValidationError as e:
    print(str(e))


1 validation error for User1
email
  value is not a valid email address (type=value_error.email)


In [19]:
# Invalid URL

try:
    User1(email="jdoe@example.com", website="jdoe")
except ValidationError as e:
    print(str(e))


1 validation error for User1
website
  invalid or missing URL scheme (type=value_error.url.scheme)


In [20]:
# Valid email

user = User1(email="jdoe@example.com", website="https://www.example.com")
user # the URL is parsed into an object, giving access to the different parts of it


User1(email='jdoe@example.com', website=HttpUrl('https://www.example.com', scheme='https', host='www.example.com', tld='com', host_type='domain'))

### Adding custom data validation with Pydantic

A pydantic model that applies a custom validation function at the field level

In [21]:
# A person with an invalid birthdate

try:
    Person5(first_name="John", last_name="Doe", birthdate="1800-01-01")
except ValidationError as e:
    print(str(e))



1 validation error for Person5
birthdate
  You seem a bit too old! (type=value_error)


In [22]:
# A person with a valid birthdate

person = Person5(first_name="John", last_name="Doe", birthdate="1991-01-01")
person


Person5(first_name='John', last_name='Doe', birthdate='Your age is 31')

A pydantic model that applies a custom validation function at the object level

In [23]:
# A registration with the passwords not matching

try:
    UserRegistration(
        email="jdoe@example.com", 
        password="aa", 
        password_confirmation="bb"
    )
except ValidationError as e:
    print(str(e))


1 validation error for UserRegistration
__root__
  Passwords don't match (type=value_error)


In [24]:
# A valid registration

user_registration = UserRegistration(
    email="jdoe@example.com", 
    password="aa", 
    password_confirmation="aa"
)
user_registration


UserRegistration(email='jdoe@example.com', password='aa', password_confirmation='aa')

A pydantic model with a custom validator function that runs before pydantic parsing

In [25]:
m = ModelWithEarlyValidation(values="1,2,3")
m.values


[1, 2, 3]

### Working with Pydantic objects

Converting a model to a dict

In [26]:
person = Person3(
    first_name="John",
    last_name="Doe",
    gender=Gender.MALE,
    birthdate="1991-01-01",
    interests=["travel", "sports"],
    address={
        "street_address": "12 Squirell Street",
        "postal_code": "424242",
        "city": "Woodtown",
        "country": "US",
    },
)


In [27]:
# Just call the 'dict()' method, as follows

person_dict = person.dict() 
print(person_dict["first_name"]) # "John"
print(person_dict["address"]["street_address"])


John
12 Squirell Street


In [28]:
# We can also include or exclude some fields, as follows

person_include = person.dict(include={"first_name", "last_name"})
print(person_include)

person_exclude = person.dict(exclude={"birthdate", "interests"})
print(person_exclude)


{'first_name': 'John', 'last_name': 'Doe'}
{'first_name': 'John', 'last_name': 'Doe', 'gender': <Gender.MALE: 'MALE'>, 'address': {'street_address': '12 Squirell Street', 'postal_code': '424242', 'city': 'Woodtown', 'country': 'US'}}


In [29]:
# For nested dicts, we can use a dictionary to specify which sub-field 
# to include or exclude, as follows

person_nested_include = person.dict(
    include={
        "first_name": ..., # Notice the use of the ellipsis for all the values
        "last_name": ...,
        "address": {"city", "country"},
    }
)
print(person_nested_include)


{'first_name': 'John', 'last_name': 'Doe', 'address': {'city': 'Woodtown', 'country': 'US'}}


In [30]:
# In case of a commonly used conversion to dict, it can be put 
# in a method inside the model, as follows

person = Person6(
    first_name="John",
    last_name="Doe",
    gender=Gender.MALE,
    birthdate="1991-01-01",
    interests=["travel", "sports"],
    address={
        "street_address": "12 Squirell Street",
        "postal_code": "424242",
        "city": "Woodtown",
        "country": "US",
    },
)

name_dict = person.name_dict()
name_dict


{'first_name': 'John', 'last_name': 'Doe'}

In [1]:
# For the last 2 examples, start the following server in a separate CLI

"""
!uvicorn chapter4_endpoints:app
"""


'\n!uvicorn chapter4_endpoints:app\n'

In [2]:
LOCALHOST = 'http://localhost:8000'


In [3]:
# Creating a pydantic instance 'PostDB' with a new ID field and using the dict version 
# of the input 'PostCreate' instance for the rest of the fields

!http POST "{LOCALHOST}/posts" title="mytitle" content="mycontent"

HTTP/1.1 201 Created
content-length: 48
content-type: application/json
date: Tue, 05 Jul 2022 14:18:38 GMT
server: uvicorn

{
    "content": "mycontent",
    "id": 1,
    "title": "mytitle"
}



In [4]:
# Patching a pydantic instance 'PostDB' with new values of the 'title' and 'content' fields,
# derived from a dict version of the input 'PostPartialUpdate' instance

!http PATCH "{LOCALHOST}/posts/2" title="mytitle2" content="mycontent2"


HTTP/1.1 200 OK
content-length: 50
content-type: application/json
date: Tue, 05 Jul 2022 14:18:39 GMT
server: uvicorn

{
    "content": "mycontent2",
    "id": 2,
    "title": "mytitle2"
}

